In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
T = 4
Delta = 500
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]
print(cp_truth)

n = 10

[ 500 1000 1500]


In [5]:
path = '/content/drive/MyDrive/CPD_BT/experiment_random/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    beta_list = np.load(f)
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [6]:
X_train_list.shape

(100, 2000, 10)

In [7]:
np.random.seed(0)

m_intervals = 50
grid_n = 90
gamma_list = [20, 40]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_wbs = np.zeros(B)
loc_error_wbs = np.zeros(B)
K_wbs = np.zeros(B)

cp_best_list = []
param_best_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    wbs_fit = wbs_cv_bt(m_intervals, grid_n, lam_list, gamma_list, smooth = 5, buffer = 5)
    res = wbs_fit.fit((X_train, Y_train), (X_test, Y_test))
    cp_best, cp_val, cusum_val, threshold_best, grid = res   
    run_time_wbs[b] = time.time() - start_time
    loc_error_wbs[b] = cp_distance(cp_best, cp_truth)
    K_wbs[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(threshold_best)

    print(b)

print('---------- wbs -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_wbs.mean(), run_time_wbs.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_wbs.std(), run_time_wbs.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_wbs < T - 1), sum(K_wbs == T - 1), sum(K_wbs > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- wbs -----------
avg loc error: 94.92, avg time: 33.60505405426025
std loc error: 174.7820173816517, std time: 5.369333548649702
K < K*: 16, K = K*: 84, K > K*: 0


In [8]:
loc_error_wbs

array([512.,  16.,  16.,  10.,  16.,  16.,  10.,  10.,  16.,  16.,  26.,
       490.,  12.,  18., 512.,  16.,  54.,  12.,  32.,  16., 490.,  10.,
       494.,  10.,  12.,  12.,  16.,  32.,  18.,  12.,  18.,  26.,  12.,
        12.,  12.,  16., 494.,  32.,  10.,  60.,  10.,  18.,  16.,  12.,
       474.,  10.,  10.,  72.,  10.,  56.,  10.,  10.,  12.,  10.,  12.,
        16.,  10.,  28.,  18.,  38.,  12., 490.,  10.,  16.,  34., 494.,
        32., 496.,  10.,  12.,  10., 496.,  16.,  18.,  16.,  18., 100.,
       490.,  10.,  12.,  12., 490.,  10.,  12.,  10.,  16.,  18., 494.,
        16.,  38.,  16.,  18.,  12.,  12., 494.,  16.,  10.,  18.,  16.,
       496.])

In [9]:
cp_best_list

[[1012, 1496],
 [484, 1012, 1496],
 [484, 990, 1496],
 [506, 990, 1496],
 [484, 990, 1496],
 [484, 1012, 1496],
 [506, 990, 1496],
 [506, 990, 1496],
 [484, 1012, 1496],
 [484, 1012, 1496],
 [484, 990, 1474],
 [990, 1496],
 [506, 1012, 1496],
 [506, 990, 1518],
 [1012, 1496],
 [484, 990, 1496],
 [506, 946, 1496],
 [506, 1012, 1496],
 [484, 968, 1496],
 [484, 1012, 1496],
 [990, 1496],
 [506, 990, 1496],
 [506, 1496],
 [506, 990, 1496],
 [506, 1012, 1496],
 [506, 1012, 1496],
 [484, 1012, 1496],
 [484, 968, 1518],
 [506, 990, 1518],
 [506, 1012, 1496],
 [484, 1012, 1518],
 [506, 990, 1474],
 [506, 1012, 1496],
 [506, 1012, 1496],
 [506, 1012, 1496],
 [484, 990, 1496],
 [506, 1496],
 [506, 968, 1496],
 [506, 990, 1496],
 [440, 990, 1496],
 [506, 990, 1496],
 [506, 1012, 1518],
 [484, 990, 1496],
 [506, 1012, 1496],
 [484, 1474],
 [506, 990, 1496],
 [506, 990, 1496],
 [572, 1012, 1518],
 [506, 990, 1496],
 [506, 1056, 1496],
 [506, 990, 1496],
 [506, 990, 1496],
 [506, 1012, 1496],
 [506,

In [10]:
import pickle
with open(path + 'wbs_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, loc_error_wbs, run_time_wbs, K_wbs], f)